In [2]:
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/gdrive')

ValueError: mount failed

In [ ]:
# Dataset paths
dataset_dir = "/content/gdrive/My Drive/Vehicles"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")
output_dir = "/content/yolo_dataset_v"

# YOLO formatted paths
images_train_dir = os.path.join(output_dir, "images/train")
images_test_dir = os.path.join(output_dir, "images/test")
labels_train_dir = os.path.join(output_dir, "labels/train")
labels_test_dir = os.path.join(output_dir, "labels/test")

# Create YOLO dataset directories
os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(images_test_dir, exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_test_dir, exist_ok=True)

# Classes and Mapping
classes = ["Big Truck", "City Car", "Multi Purpose Vehicle", "Sedan", "Sport Utility Vehicle", "Truck", "Van"]
class_map = {cls: idx for idx, cls in enumerate(classes)}

In [ ]:
def process_dataset(input_dir, images_output_dir, labels_output_dir, class_map):
    """
    Process dataset by inferring labels from folder names and creating YOLO format annotations.
    """
    for class_name, class_id in class_map.items():
        class_folder = os.path.join(input_dir, class_name)
        if not os.path.exists(class_folder):
            raise FileNotFoundError(f"Folder {class_folder} does not exist!")

        for file_name in os.listdir(class_folder):
            if file_name.endswith((".jpg", ".png")):
                # Copy image to YOLO images folder
                source_path = os.path.join(class_folder, file_name)
                dest_path = os.path.join(images_output_dir, file_name)
                shutil.copy(source_path, dest_path)

                # Create label file in YOLO format
                label_file = file_name.replace(".jpg", ".txt").replace(".png", ".txt")
                label_path = os.path.join(labels_output_dir, label_file)
                with open(label_path, "w") as f:
                    # Dummy bounding box: full image as object
                    f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # Update with real bbox if available

# Process training and testing datasets
process_dataset(train_dir, images_train_dir, labels_train_dir, class_map)
process_dataset(test_dir, images_test_dir, labels_test_dir, class_map)


In [ ]:
# Create a dataset YAML configuration file
yaml_path = os.path.join(output_dir, "custom_dataset.yaml")
yaml_content = f"""
path: {output_dir}
train: images/train
val: images/test
nc: {len(classes)}
names: {classes}
"""

with open(yaml_path, "w") as yaml_file:
    yaml_file.write(yaml_content)


In [ ]:
# Install YOLOv8
!pip install ultralytics

from ultralytics import YOLO

# Load YOLOv8 Pretrained Model
model = YOLO("yolov8n.pt")  # Use YOLOv8 Nano for lightweight training

# Train the Model
model.train(
    data=yaml_path,  # Path to dataset configuration file
    epochs=200,       # Number of epochs
    imgsz=640,       # Image size
    batch=16,        # Batch size
    workers=2,       # CPU workers for Colab
    project="yolo_v8_project",  # Save results here
    name="training_results",    # Subfolder for this training session
    pretrained=True   # Use pretrained weights
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 15.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 17.7MB/s]


Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/yolo_dataset_v/custom_dataset.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=yolo_v8_project, name=training_results, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

100%|██████████| 755k/755k [00:00<00:00, 3.41MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all       4731       4731      0.224      0.705      0.196      0.193



200 epochs completed in 6.879 hours.
Optimizer stripped from yolo_v8_project/training_results/weights/last.pt, 6.3MB
Optimizer stripped from yolo_v8_project/training_results/weights/best.pt, 6.3MB

Validating yolo_v8_project/training_results/weights/best.pt...
Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 148/148 [00:23<00:00,  6.34it/s]


                   all       4731       4731      0.989      0.987      0.994      0.994
             Big Truck        173        173      0.992          1      0.995      0.995
              City Car       1427       1427      0.986      0.972      0.993      0.993
 Multi Purpose Vehicle        794        794      0.976      0.976      0.992      0.992
                 Sedan        795        795      0.984      0.992      0.994      0.994
 Sport Utility Vehicle        547        547      0.989      0.968      0.994      0.994
                 Truck        345        345      0.999      0.997      0.995      0.995
                   Van        650        650          1          1      0.995      0.995
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to yolo_v8_project/training_results


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a60ece7cc70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [ ]:
import torch

In [ ]:
# Save the model using torch.save

torch.save(model.state_dict(), 'yolo_model_weights.pth')

In [ ]:
# Validate the model
results = model.val()
print("Validation Results:", results)


Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/yolo_dataset_v/labels/test.cache... 4731 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4731/4731 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 296/296 [00:30<00:00,  9.73it/s]


                   all       4731       4731      0.989      0.987      0.994      0.994
             Big Truck        173        173      0.992          1      0.995      0.995
              City Car       1427       1427      0.986      0.972      0.993      0.993
 Multi Purpose Vehicle        794        794      0.976      0.976      0.992      0.992
                 Sedan        795        795      0.984      0.992      0.994      0.994
 Sport Utility Vehicle        547        547      0.989      0.968      0.994      0.994
                 Truck        345        345      0.999      0.997      0.995      0.995
                   Van        650        650          1          1      0.995      0.995
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to yolo_v8_project/training_results2
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metri

In [ ]:
# Export the model to ONNX or other formats if needed
model.export(format="onnx")  # Options: onnx, torchscript, coreml, etc.


Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'yolo_v8_project/training_results/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.42...
ONNX: export success ✅ 1.0s, saved as 'yolo_v8_project/training_results/weights/best.onnx' (11.7 MB)

Export complete (1.1s)
Results saved to /content/yolo_v8_project/training_results/weights
Predict:         yolo predict task=detect model=yolo_v8_project/training_results/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo_v8_project/training_results/weights/best.onnx imgsz=640 data=/content/yolo_dataset_v/custom_dataset.yaml  
Visualize:       https://netron.app


'yolo_v8_project/training_results/weights/best.onnx'